In [49]:
import numpy as np
from typing import List, Dict, Union, Tuple
from tenpy.networks.site import SpinSite
from tenpy.linalg import np_conserved as npc
from tenpy.linalg.np_conserved import Array
from tenpy.networks.mpo import MPO
from tenpy.models.model import MPOModel
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS  
from tenpy.algorithms.network_contractor import ncon

In [50]:
def random_L(d: int, Dl: int, Dr: int, seed=None) -> np.ndarray:
    """
    Tensore L_k di shape (d, Dl, Dr).
    Su ogni riga non nulla c’è un solo '1' in colonna casuale.
    Ogni riga (x, i) ha probabilità 1/(Dr+1) di essere tutta zero.
    """
    rng = np.random.default_rng(seed)
    L = np.zeros((d, Dl, Dr), dtype=np.uint8)

    # J[x, i] ∈ {0, ..., Dr} con distribuzione uniforme
    J = rng.integers(0, Dr + 1, size=(d, Dl))
    mask = (J != Dr)  # True dove mettiamo un '1'

    xs, is_ = np.where(mask)
    js = J[xs, is_]
    L[xs, is_, js] = 1

    return L


def random_R(d: int, Dl: int, Dr: int, seed=None) -> np.ndarray:
    """
    Tensore R_k di shape (d, Dl, Dr).
    Su ogni colonna non nulla c’è un solo '1' in riga casuale.
    Ogni colonna (x, j) ha probabilità 1/(Dl+1) di essere tutta zero.
    """
    rng = np.random.default_rng(seed)
    R = np.zeros((d, Dl, Dr), dtype=np.uint8)

    # I[x, j] ∈ {0, ..., Dl} con distribuzione uniforme
    I = rng.integers(0, Dl + 1, size=(d, Dr))
    mask = (I != Dl)

    xs, js = np.where(mask)
    is_ = I[xs, js]
    R[xs, is_, js] = 1

    return R


def random_A1(d: int, D: int, seed=None) -> np.ndarray:
    """
    Primo sito: shape (d, 1, D). Binario libero.
    """
    rng = np.random.default_rng(seed)
    # Estraggo in {0, 1}, uniforme, direttamente in uint8
    return rng.integers(0, 2, size=(d, 1, D), dtype=np.uint8)

def wrap_site_tensor(T: np.ndarray):
    """(d, Dl, Dr) -> npc.Array labels ['vL','p','vR'] shape (Dl, d, Dr)"""
    Dl = T.shape[1]
    Ai = np.transpose(T, (1, 0, 2))  # (Dl, d, Dr)
    return npc.Array.from_ndarray_trivial(Ai, labels=['vL', 'p', 'vR'])

def tenpy_sites_and_svs(d: int, right_dims):
    N = len(right_dims)
    S = (d - 1) / 2.0
    site = SpinSite(S=S, conserve=None)
    lattice = Chain(L=N, site=site)
    sites = lattice.mps_sites()
    svs = [np.ones(1, dtype=float)]
    svs += [np.ones(right_dims[i], dtype=float) for i in range(N - 1)]
    svs += [np.ones(1, dtype=float)]
    return sites, svs

In [51]:
def site_tensor_from_Bi(Bi: dict, d: int, *, strict_keys: bool = True):
    """
    Costruisce T[x] = Bi[x] con fallback a zeri. T shape: (d, Dl, Dr).

    Se strict_keys=True, controlla che tutte le chiavi di Bi siano in [0, d-1].
    Richiede che Bi non sia vuoto.
    """
    if not Bi:
        raise ValueError("Bi non può essere vuoto: serve almeno una matrice per fissare (Dl, Dr).")

    if strict_keys:
        bad_keys = [k for k in Bi.keys() if not (0 <= k < d)]
        if bad_keys:
            raise ValueError(f"Chiavi fuori range [0, {d-1}]: {bad_keys}")

    # tutte le matrici devono avere la stessa shape 2D
    shapes = {np.shape(M) for M in Bi.values()}
    if len(shapes) != 1:
        raise ValueError(f"shape incoerenti in Bi: {shapes}")
    Dl, Dr = next(iter(shapes))
    if len((Dl, Dr)) != 2:
        raise ValueError(f"Le matrici in Bi devono essere 2D, shape trovata: {Dl, Dr}")

    # dtype comune (considera anche i fallback a 0)
    dtype = np.result_type(*[np.asarray(Bi.get(x, 0)).dtype for x in range(d)])

    T = np.zeros((d, Dl, Dr), dtype=dtype)
    zero_block = np.zeros((Dl, Dr), dtype=dtype)

    for x in range(d):
        Mx = np.asarray(Bi.get(x, zero_block), dtype=dtype)
        if Mx.shape != (Dl, Dr):
            raise ValueError(f"Bi[{x}] ha shape {Mx.shape}, atteso {(Dl, Dr)}")
        T[x] = Mx

    return T, Dl, Dr

def build_mps(B_list, d: int):
    # Controllo d intero positivo
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    d = int(d)

    N = len(B_list)
    if N == 0:
        raise ValueError("B_list non può essere vuoto.")

    tensors = []
    right_dims = []

    prev_Dr = None

    for i, Bi in enumerate(B_list):
        T, Dl, Dr = site_tensor_from_Bi(Bi, d)

        # Bordo sinistro
        if i == 0:
            if Dl != 1:
                raise ValueError(f"sito {i}: Dl deve essere 1 (bordo sinistro), trovato {Dl}")
        else:
            # Bond interno: Dl(i) deve coincidere con Dr(i-1)
            if Dl != prev_Dr:
                raise ValueError(
                    f"mismatch bond interno tra sito {i-1} e {i}: "
                    f"Dr[{i-1}]={prev_Dr}, Dl[{i}]={Dl}"
                )

        # Bordo destro
        if i == N - 1 and Dr != 1:
            raise ValueError(f"sito {i}: Dr deve essere 1 (bordo destro), trovato {Dr}")

        tensors.append(T)
        right_dims.append(Dr)
        prev_Dr = Dr

    A = [wrap_site_tensor(T) for T in tensors]
    sites, svs = tenpy_sites_and_svs(d, right_dims)

    return MPS(sites, A, svs, bc='finite', form='A')

In [52]:
def build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu):
    pmid = 1 - pd - pu
    if not (0 <= pd <= 1 and 0 <= pu <= 1 and pd + pu <= 1):
        raise ValueError("Probabilità non valide: servono pd, pu >=0 e pd+pu <= 1")

    B_list = []

    # Sito 1: (1,2)
    B1 = {
        0: np.array([[ (S0/(N+1))*d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ]]),
        1: np.array([[ (S0/(N+1))*m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ]]),
        2: np.array([[ (S0/(N+1))*u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ]]),
    }
    B_list.append(B1)

    # Siti 2..N-1: (2,2)  ← QUI MANCANO NEL TUO FILE
    for i in range(2, N):
        Bi = {
            0: np.array([[ d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ],
                         [     pd,                               1 ]]),
            1: np.array([[ m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ],
                         [      pmid,                               1 ]]),
            2: np.array([[ u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ],
                         [    pu,                                 1 ]]),
        }
        B_list.append(Bi)

    # Sito N: (2,1)
    BN = {
        0: np.array([[ d_op*pd ],
                     [     pd ]],),
        1: np.array([[ m_op*pmid ],
                     [      pmid ]]),
        2: np.array([[ u_op*pu ],
                     [     pu ]]),
    }
    B_list.append(BN)
    return B_list

In [53]:
def build_mps_AR(d: int, N: int, D: int, seed=None):
    """
    Crea b(x) con bond dimensione D:
      A1: (d, 1, D)
      R2..R_{N-1}: (d, D, D)
      RN: (d, D, 1)
    """
    # controlli basilari
    if N < 2:
        raise ValueError("N deve essere >= 2 per avere A1 e almeno un R.")
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    if D < 1 or int(D) != D:
        raise ValueError("`D` deve essere intero positivo.")
    d = int(d)
    D = int(D)

    A1 = random_A1(d, d, seed=seed)

    Rs = []
    for k in range(2, N):
        sk = None if seed is None else seed + k
        Rs.append(random_R(d, Dl=D, Dr=D, seed=sk))

    sN = None if seed is None else seed + N
    RN = random_R(d, Dl=D, Dr=1, seed=sN)

    tensors = [A1] + Rs + [RN]

    # wrap TenPy
    A = [wrap_site_tensor(T) for T in tensors]
    right_dims = [T.shape[2] for T in tensors]  # Dr per ciascun sito
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')

def build_mps_AR_bond(d: int, N: int, D: int, seed=None):
    """
    Crea un MPS psi con N siti (N dispari) e bond che crescono verso il centro
    come Dl -> min(Dl * d, D) e poi decrescono in modo simmetrico.

    Tensore sito i (0-based) ha shape (d, Dl_i, Dr_i).
    """
    # controlli basilari
    if N < 3 or N % 2 == 0:
        raise ValueError("N deve essere dispari e >= 3.")
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    if D < 1 or int(D) != D:
        raise ValueError("`D` deve essere intero positivo.")
    d = int(d)
    D = int(D)

    # half = indice del bond centrale
    half = N // 2  # per N=7 -> 3

    # costruiamo i bond da sinistra fino al centro
    bonds = [1]   # bond[0] = 1
    Dl = 1
    for k in range(1, half + 1):
        candidate = Dl * d
        Dr = candidate if candidate < D else D
        bonds.append(Dr)
        Dl = Dr

    # riflettiamo per avere simmetria: [1, ..., centro, ..., 1]
    # es: [1,3,9,10] -> [1,3,9,10,10,9,3,1]
    bonds = bonds + bonds[::-1]
    assert len(bonds) == N + 1  # N siti -> N+1 bond

    tensors = []

    # primo sito: Dl = bonds[0]=1, Dr = bonds[1]
    Dr0 = bonds[1]
    A1 = random_A1(d, Dr0, seed=seed)   # (d,1,Dr0)
    tensors.append(A1)

    # siti 1..N-1
    for i in range(1, N):
        Dl_i = bonds[i]
        Dr_i = bonds[i + 1]
        sk = None if seed is None else seed + i
        tensors.append(random_R(d, Dl=Dl_i, Dr=Dr_i, seed=sk))  # (d, Dl_i, Dr_i)

    # wrap TenPy
    A = [wrap_site_tensor(T) for T in tensors]  # (d,Dl,Dr) -> (Dl,d,Dr)
    right_dims = [T.shape[2] for T in tensors]  # Dr per sito
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')




In [54]:
S0, K, N = 1.0, 0.2, 9
d_op, m_op, u_op = 0.9, 1.0, 3
pd, pu = 0.2, 0.4   

B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)

b = build_mps(B_list, d=3)

psi = build_mps_AR_bond(d=3, N=len(B_list), D=9)

val = b.overlap(psi)
print("sum_x psi(x)b(x) =", val)

sum_x psi(x)b(x) = -56.942908561416786


In [55]:
def contract_left(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i-1)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == 1:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")


    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # A0: (-1, -2,  1)
    # A1: ( 1, -3,  2)
    # A2: ( 2, -4,  3)
    # ...
    # A_{i-2}: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d

    return T

def contract_right(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i,N)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == N:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")

    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # Ai: (-1, -2,  1)
    # Ai+1: ( 1, -3,  2)
    # Ai+2: ( 2, -4,  3)
    # ...
    # AN-1: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d
    return T

UL = contract_left(tensor=psi,i=3,N=len(B_list))
UR = contract_right(tensor=psi,i=3,N=len(B_list))


In [56]:
def tensorial_derivative(psi, b, site):

    N_psi = psi.L
    N_b  = b.L
    if N_psi != N_b:
        raise ValueError(f"psi e b hanno lunghezze diverse: {N_psi} vs {N_b}")
    N = N_psi

    if not (1 <= site <= N):
        raise ValueError(f"site={site} fuori range [1, {N}]")
    
    UL = contract_left(tensor=psi,i=site,N=N)
    UR = contract_right(tensor=psi,i=site,N=N)
    BL = contract_left(tensor=b,i=site,N=N)
    BR = contract_right(tensor=b,i=site,N=N)

    if UL is not None and len(UL.shape) != len(BL.shape):
        raise ValueError(f"Dimensione di UL diversa da BL")
    if UR is not None and len(UR.shape) != len(BR.shape):
        raise ValueError(f"Dimensione di UR diversa da BR")

    if UL is not None:
        left_tensors = [UL,BL]
        left_links = [
            [-1] + [k for k in range(1,site)] + [-2], 
        [-3] + [k for k in range(1,site)] + [-4] 
        ]
        #print('left contraction links',left_links)
        left_contraction = ncon(left_tensors,left_links) 
    ## Il risultato è un tensore di dimensione (1,D_i;1,2)

    if UR is not None:
        right_tensors = [UR,BR]
        right_links = [
            [-1] + [k for k in range(1,N-site+1)] + [-2], 
        [-3] + [k for k in range(1,N-site+1)] + [-4] 
        ]
        #print('right contraction links',right_links)
        right_contraction = ncon(right_tensors,right_links) 
    ## Il risultato è un tensore di dimensione (D_i+1,1;2,1)

    if site > 1 and site < N:
        final_tensors = [left_contraction, b.get_B(site-1),right_contraction]
        final_links = [
            [-1] + [-2] + [-3] + [1],
            [1] + [-4] + [2],
            [-5] + [-6] + [2] + [-7]
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    elif site == 1:
        final_tensors = [b.get_B(site-1),right_contraction]
        final_links = [
            [-1] + [-2] + [1],
            [-3] + [-4] + [1] + [-5]
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    elif site == N:
        final_tensors = [left_contraction, b.get_B(site-1)]
        final_links = [
            [-1] + [-2] + [-3] + [1],
            [1] + [-4] + [-5],
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    return np.squeeze(final_contraction)  

In [57]:
derivata = tensorial_derivative(psi=psi,b=b, site=9)
print(derivata.shape)

(3, 3)


In [58]:
def Greedy(A: np.ndarray) -> np.ndarray:
    """
    Greedy compression of the rows of A.

    Parameters
    ----------
    A : np.ndarray
        Input matrix (2D).

    Returns
    -------
    np.ndarray
        Matrix L encoding the greedy merges/deletions of rows of A.
    """
    if A.ndim != 2:
        raise ValueError("A deve essere una matrice 2D")

    M = A.copy()
    L = np.identity(M.shape[0])

    while L.shape[1] > A.shape[1]:
        n_rows = M.shape[0]

        # Row-wise positive sums
        row_sums = np.zeros(n_rows)
        for i in range(n_rows):
            row = M[i]
            row_sums[i] = row[row > 0].sum()

        # Gain from merging each pair of rows
        merge_gain = np.full((n_rows, n_rows), -np.inf, dtype=float)
        for i in range(n_rows):
            for j in range(n_rows):
                if i == j:
                    continue
                merged_row = M[i] + M[j]
                pos_sum = merged_row[merged_row > 0].sum()
                merge_gain[i, j] = pos_sum - row_sums[i] - row_sums[j]

        # Greedy choice: best row to drop vs best pair to merge
        ropt = np.argmin(row_sums)
        iopt, jopt = np.unravel_index(np.argmax(merge_gain), merge_gain.shape)

        if merge_gain[iopt, jopt] > -row_sums[ropt]:
            # Merge rows iopt and jopt
            M[iopt] = M[iopt] + M[jopt]
            M = np.delete(M, jopt, axis=0)

            L[:, iopt] = L[:, iopt] + L[:, jopt]
            L = np.delete(L, jopt, axis=1)
        else:
            # Drop row ropt
            M = np.delete(M, ropt, axis=0)
            L = np.delete(L, ropt, axis=1)

    return L

In [59]:
def SweepingAlgorithm(b, d, D, err):
    N = b.L
    psi = build_mps_AR_bond(d=d, N=N, D=D)

    # overlap iniziale
    k_ref = b.overlap(psi)

    # primo update sul sito 1 (come prima)
    site = 1
    A_tilde = tensorial_derivative(psi=psi, b=b, site=site)
    A_prime = A_tilde.to_ndarray()
    L_out = Greedy(A_prime)
    d_phys, Dr = L_out.shape
    L = L_out.reshape(1, d_phys, Dr)
    L_tenpy = npc.Array.from_ndarray_trivial(L, labels=['vL', 'p', 'vR'])
    psi.set_B(site - 1, L_tenpy)

    nstep = 1
    updates_since_check = 1   # abbiamo già fatto un update
    K = 4        # o N, 2*N, ecc.
    max_steps = 100000

    while nstep < max_steps:

        # sweep left -> right
        for i in range(N - 1):
            site = i + 1
            if site == 1 and nstep == 1:
                continue

            A_prime = tensorial_derivative(psi=psi, b=b, site=site).to_ndarray()

            if 1 < site < N:
                Dl, d_phys, Dr = A_prime.shape
                L = Greedy(A_prime.reshape(Dl * d_phys, Dr)).reshape(Dl, d_phys, Dr)
            else:
                d_phys, Dr = A_prime.shape
                L =  Greedy(A_prime).reshape(1, d_phys, Dr)

            psi.set_B(i, npc.Array.from_ndarray_trivial(L, labels=['vL', 'p', 'vR']))

            nstep += 1
            updates_since_check += 1

            if updates_since_check >= K:
                k_curr = b.overlap(psi)
                print(k_curr)
                if np.abs(k_curr - k_ref) <= err * max(np.abs(k_ref), 1e-12):
                    return psi, k_curr, nstep
                k_ref = k_curr
                updates_since_check = 0

        # sweep right -> left
        for j in range(N - 1, 0, -1):
            site = j + 1
            A_prime = tensorial_derivative(psi=psi, b=b, site=site).to_ndarray()

            if 1 < site < N:
                Dl, d_phys, Dr = A_prime.shape
                R = Greedy(A_prime.reshape(Dl, Dr * d_phys).T).T.reshape(Dl, d_phys, Dr)
            else:
                Dl, d_phys = A_prime.shape
                R = Greedy(A_prime.T).T.reshape(Dl, d_phys, 1)

            psi.set_B(j, npc.Array.from_ndarray_trivial(R, labels=['vL', 'p', 'vR']))

            nstep += 1
            updates_since_check += 1

            if updates_since_check >= K:
                k_curr = b.overlap(psi)
                print(k_curr)
                if np.abs(k_curr - k_ref) <= err * max(np.abs(k_ref), 1e-12):
                    return psi, k_curr, nstep
                k_ref = k_curr
                updates_since_check = 0

    raise RuntimeError("SweepingAlgorithm did not converge within max_steps")


In [60]:
N = 15
B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)
b = build_mps(B_list, d=3)
psi , K, nstep = SweepingAlgorithm(b=b, d=3, D=10, err=1e-18)
print(nstep)

-134.62963745668674
-3.860883739476491
0.01310423983241836
0.8408397035120824
0.4592706102186114
-0.8333596695366894
-29.803243001788083
-1.3618217085281632
0.10370756581560503
0.20953092460885991
0.5249079298390966
-1.9780765124488928
-1.9156610091312056
-18.5427358149518
-2.1989244193678896
-1.238899168066721
-0.22260677835899018
0.8349167390059724
0.49020289187282984
-0.8964775934582278
-18.479536413945567
-0.8516053418777942
-2.3398003203922215
-0.2708200347008948
0.6210105668018493
-0.17242597833719597
-2.9556153334143875
-18.479536413945567
-3.348307907981714
-0.7507862468414739
-0.10137965055936277
0.6487517531136339
-3.574694101962434
-0.8432361875246106
-22.512130809836265
-1.002148462930361
-1.3896091468393375
-0.41094544864048665
0.6867207562919061
0.29687265894211545
-3.697827440274075
6.0317389478404975
-5.503041815424384
-0.6097242814707149
-0.5352330788505237
0.8991643368744875
0.1620661798423053
-2.408117851061025
-21.179393909259485
-1.9569030595936925
-0.1825788965133

KeyboardInterrupt: 